In [2]:
!pip list | grep datasets

In [1]:
import sys
print(sys.executable)

/home/gridsan/mhadjiivanov/.conda/envs/g_retriever/bin/python


In [2]:
import os
import torch
import pandas as pd
from torch.utils.data import Dataset
import datasets
from tqdm import tqdm
from src.dataset.utils.retrieval import retrieval_via_pcst, retrieval_via_shortest_paths
import sys

/home/gridsan/mhadjiivanov/.conda/envs/g_retriever/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [27]:
retrieved_triples = torch.load('/home/gridsan/mhadjiivanov/meng/SubgraphRAG/retrieve/webqsp_Jan03-19:20:34/retrieval_result.pth')

In [31]:
retrieved_triples['WebQTest-0']['scored_triples']

[('Jamaican English',
  'language.human_language.countries_spoken_in',
  'Jamaica',
  0.9733461737632751),
 ('Jamaica',
  'location.country.languages_spoken',
  'Jamaican English',
  0.961194634437561),
 ('Jamaica',
  'location.country.official_language',
  'Jamaican English',
  0.9468052387237549),
 ('Jamaican Creole English Language',
  'language.human_language.countries_spoken_in',
  'Jamaica',
  0.9090040922164917),
 ('Jamaica',
  'location.country.languages_spoken',
  'Jamaican Creole English Language',
  0.8998770713806152),
 ('Jamaican Creole English Language',
  'language.human_language.main_country',
  'Jamaica',
  0.7312235236167908),
 ('Jamaica', 'royalty.kingdom.rulers', 'Elizabeth II', 0.01750544086098671),
 ('Elizabeth II', 'royalty.monarch.kingdom', 'Jamaica', 0.004011394456028938),
 ('Jamaican dollar',
  'finance.currency.countries_used',
  'Jamaica',
  0.002863228088244796),
 ('Jamaica',
  'travel.travel_destination.tourist_attractions',
  'Blue Mountains',
  0.0017535

In [51]:
import torch
import numpy as np
from torch_geometric.data.data import Data
import sys

def SubGRAG_retrieval(graph,textual_nodes, textual_edges, subgrag_triples):
    # Map string identifiers to node indices (with lowercase comparison)
    node_to_idx = {str(row['node_attr']).lower(): idx for idx, row in textual_nodes.iterrows()}
    edge_to_idx = {(int(row['src']),str(row['edge_attr']).lower(),int(row['dst'])): idx for idx, row in textual_edges.iterrows()}
    
    selected_nodes = set()
    selected_edges = set()
    for h,r,t in subgrag_triples:
        try:
            h_id = node_to_idx[h.lower()]
            t_id = node_to_idx[t.lower()]
            edge_id = edge_to_idx[(h_id,r.lower(),t_id)]
        except:
            continue
            
        selected_nodes.add(h_id)
        selected_nodes.add(t_id)
        selected_edges.add(edge_id)
        
    selected_nodes = sorted(list(selected_nodes))
    selected_edges = list(selected_edges)
    
    if not selected_nodes or not selected_edges:
        print('missing nodes or edges')
        desc = textual_nodes.to_csv(index=False) + '\n' + textual_edges.to_csv(index=False, columns=['src', 'edge_attr', 'dst'])
        graph = Data(x=graph.x, edge_index=graph.edge_index, edge_attr=graph.edge_attr, num_nodes=graph.num_nodes)
        return graph, desc

    # Create mapping for new node indices
    mapping = {n: i for i, n in enumerate(selected_nodes)}

    # Create new graph with selected nodes and edges
    x = graph.x[selected_nodes]
    edge_attr = graph.edge_attr[selected_edges]
    edge_index = graph.edge_index[:, selected_edges]
    
    # Remap node indices
    src = [mapping[i.item()] for i in edge_index[0]]
    dst = [mapping[i.item()] for i in edge_index[1]]
    edge_index = torch.LongTensor([src, dst])

    # Create description
    n = textual_nodes.iloc[selected_nodes]
    e = textual_edges.iloc[selected_edges]
    desc = n.to_csv(index=False) + '\n' + e.to_csv(index=False, columns=['src', 'edge_attr', 'dst'])

    # Create new Data object
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, num_nodes=len(selected_nodes))

    return data, desc

In [52]:
path = '/home/gridsan/mhadjiivanov/meng/G-Retriever/dataset/webqsp'
path_nodes = f'{path}/nodes'
path_edges = f'{path}/edges'
path_graphs = f'{path}/graphs'

cached_graph = f'{path}/cached_graphs'
cached_desc = f'{path}/cached_desc'

SubGRAG_cached_graph = f'{path}/SubGRAG_cached_graphs'
SubGRAG_cached_desc = f'{path}/SubGRAG_cached_desc'

In [20]:
dataset = datasets.load_dataset('/home/gridsan/mhadjiivanov/meng/G-Retriever/hf/datasets/RoG-webqsp')
dataset = datasets.concatenate_datasets([dataset['train'],dataset['validation'],dataset['test']])

In [53]:
for index in tqdm(range(len(dataset))):
    if 'Test' not in dataset[index]['id']: 
        continue
    
    nodes = pd.read_csv(f'{path_nodes}/{index}.csv')
    edges = pd.read_csv(f'{path_edges}/{index}.csv')
    graph = torch.load(f'{path_graphs}/{index}.pt')
    
    num_trips = torch.load(f'{cached_graph}/{index}.pt').edge_index.shape[1]    
    subgrag_triples = retrieved_triples[dataset[index]['id']]['scored_triples'][:num_trips]
    subgrag_triples = [tup[:3] for tup in subgrag_triples]
    
    SubGRAG_subg, SubGRAG_desc = SubGRAG_retrieval(graph, nodes, edges, subgrag_triples)
    torch.save(SubGRAG_subg, f'{SubGRAG_cached_graph}/{index}.pt')
    open(f'{SubGRAG_cached_desc}/{index}.txt', 'w').write(SubGRAG_desc)

 74%|███████▍  | 3478/4700 [04:35<11:55,  1.71it/s]

missing nodes or edges


100%|██████████| 4700/4700 [18:44<00:00,  4.18it/s]
